In [231]:
import findspark
findspark.init()
import pyspark

In [232]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import requests, json, os, sys, time, re
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import pandas as pd

In [233]:
######## Spark session is a unified entry point of a spark application  #############
spark = SparkSession \
    .builder \
    .appName('spark-rec') \
    .config("configuration_key", "configuration_value") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [234]:
import os
import math
import datetime
import pyspark.sql.functions as sf
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark import SparkConf, SparkContext
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import TimestampType
import hashlib

In [235]:
ratings_raw_data = sc.textFile("../../pre_processed_data/experiment_1_dataset.csv")
ratings_raw_data_header = ratings_raw_data.take(1)[0]
ratings_data = ratings_raw_data.filter(lambda line: line != ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(hashlib.sha1(tokens[0].encode('utf-8')).hexdigest(), 16) % (10 ** 8),tokens[1],int(float(tokens[2][:10])))).cache()

#ratings_data = ratings_data.map(lambda x: (x[0] , x[1], int(int(x[2])/10) +1.0 ))
#ratings_data = ratings_data.filter(lambda x: int(x[2]) >=2 )
######   we should assume each column here, by position [1],[2],[3] and data charactors
rddTraining, rddValidating, rddTesting = ratings_data.randomSplit([6,2,2], seed=1001)

In [236]:
ratings_data.take(10)

[(64592103, '55150', 1),
 (46027953, '12900', 1),
 (11418539, '4000', 100),
 (22844952, '363970', 10),
 (30902696, '317360', 4),
 (29257067, '65800', 1),
 (68176534, '218230', 6),
 (35300949, '240', 5),
 (5765927, '65700', 1),
 (39525566, '221100', 7)]

In [237]:
#ratings_raw_data = sc.textFile("../../pre_processed_data/experiment_1_dataset.csv")
#ratings_raw_data_header = ratings_raw_data.take(1)[0]
#ratings_data = ratings_raw_data.filter(lambda line: line != ratings_raw_data_header)\
#    .map(lambda line: line.split(",")).map(lambda tokens: (int(hashlib.sha1(tokens[0].encode('utf-8')).hexdigest(), 16) % (10 ** 8),tokens[1],int(float(tokens[2][:10])))).cache()
######   we should assume each column here, by position [1],[2],[3] and data charactors
#rddTraining, rddValidating, rddTesting = ratings_data.randomSplit([6,2,2], seed=1001)

In [238]:
#rddTraining.take(10)
#new_ratings_data = ratings_data.filter(lambda x: int(x[2]) >=2)
schema = ["user","item","rating"]
df = spark.createDataFrame(data=ratings_data, schema = schema)

df.printSchema()
df.show(truncate=False)

root
 |-- user: long (nullable = true)
 |-- item: string (nullable = true)
 |-- rating: long (nullable = true)

+--------+------+------+
|user    |item  |rating|
+--------+------+------+
|64592103|55150 |1     |
|46027953|12900 |1     |
|11418539|4000  |100   |
|22844952|363970|10    |
|30902696|317360|4     |
|29257067|65800 |1     |
|68176534|218230|6     |
|35300949|240   |5     |
|5765927 |65700 |1     |
|39525566|221100|7     |
|59586993|291410|2     |
|20441736|304930|1     |
|8742321 |4000  |8     |
|62551528|113400|4     |
|67019253|238460|1     |
|49237568|242860|3     |
|58504375|235800|2     |
|2591283 |10150 |9     |
|90966574|280790|2     |
|96131366|221040|2     |
+--------+------+------+
only showing top 20 rows



In [248]:
#df.groupBy("rating").count().show(100)
#df.registerTempTable("df");
#spark.sql("select count(*) , rating from df group by rating order by rating").show(100)

In [250]:
#[START how_far]
def howFarAreWe(model, against, sizeAgainst):
    againstNoRatings = against.map(lambda x: (int(x[0]), int(x[1])) )
    againstWiRatings = against.map(lambda x: ((int(x[0]),int(x[1])), int(x[2])) )
    predictions = model.predictAll(againstNoRatings).map(lambda p: ( (p[0],p[1]), p[2]) )
    predictionsAndRatings = predictions.join(againstWiRatings).values()    
    return sqrt(predictionsAndRatings.map(lambda s: (s[0] - s[1]) ** 2).reduce(add) / float(sizeAgainst))
#[END how_far]

In [253]:
#finding best set of parameters
ranks  = [5,10,15,20]
reguls = [0.1, 1,10]
iters  = [5,10,20]
alpha = [10, 20, 40]

finalModel = None
finalRank  = 0
finalRegul = float(0)
finalIter  = -1
finalDist   = float(300)
finalAlpha = float(0)

#[START train_model]
for cRank, cRegul, cIter, cAlpha in itertools.product(ranks, reguls, iters, alpha):
    model = ALS.trainImplicit(rddTraining, cRank, cIter, float(cRegul),alpha=float(cAlpha))
    dist = howFarAreWe(model, rddTraining, rddValidating.count())
    if dist < finalDist:
        print(cIter, cRank,cAlpha,cRegul)
        print("Best so far:%f" % dist)
        finalModel = model
        finalRank  = cRank
        finalRegul = cRegul
        finalIter  = cIter
        finalDist  = dist
        finalAlpha  = cAlpha 
#[END train_model]

print("Rank %i" % finalRank) 
print("Regul %f" % finalRegul) 
print("Iter %i" % finalIter)  
print("Dist %f" % finalDist) 
print("Alpha %f" % finalAlpha)

5 5 10 0.1
Best so far:32.839950
5 5 20 0.1
Best so far:32.803959
5 5 40 0.1
Best so far:32.781413
10 5 40 0.1
Best so far:32.777297
20 5 40 0.1
Best so far:32.776228
5 10 40 0.1
Best so far:32.772212
10 10 40 0.1
Best so far:32.768779


Py4JJavaError: An error occurred while calling o8493.trainImplicitALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 34072.0 failed 1 times, most recent failure: Lost task 7.0 in stage 34072.0 (TID 28280, mukeshs-mbp.attlocal.net, executor driver): java.lang.StackOverflowError
	at java.base/java.io.ObjectInputStream$PeekInputStream.peek(ObjectInputStream.java:2778)
	at java.base/java.io.ObjectInputStream$BlockDataInputStream.peek(ObjectInputStream.java:3105)
	at java.base/java.io.ObjectInputStream$BlockDataInputStream.peekByte(ObjectInputStream.java:3115)
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1814)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2115)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1646)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2410)
	at java.base/java.io.ObjectInputStream.readSerialData(

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52461)
Traceback (most recent call last):
  File "/Users/mukesh/anaconda3/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/mukesh/anaconda3/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/mukesh/anaconda3/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/mukesh/anaconda3/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/opt/apache-

In [240]:
rank = 10   
numIterations = 10
#Iteration numbers
alpha=0.01
#model = ALS.trainImplicit(rddTraining, 10, 10,alpha=0.01) #18.99407447555247
#model = ALS.trainImplicit(rddTraining, 10, 10,alpha=0.5) #18.88809845032351
#model = ALS.train(rddTraining, 10, 10) #29.13044908080498
model = ALS.trainImplicit(rddTraining, 10, 10, alpha=0.01) #18.96886550953952
testset = sc.parallelize([(80937808, 386360), (80937808, 380600)])   
##########   you can assume any user_id and any item(game) id here   ######################
model.predictAll(testset).collect()

[Rating(user=80937808, product=386360, rating=0.17942816109131407),
 Rating(user=80937808, product=380600, rating=0.05183744885757856)]

In [241]:
# Calculate all predictions
rddTesting_map = rddTesting.map(lambda r: ((r[0], r[1]))) 
predictions = model.predictAll(rddTesting_map).map(lambda r: ((r[0], r[1]), (r[2]))) 
predictions.take(5)    ####### Output 5 results
#model.predictAll(rddTesting_map).collect()     Show all the Recommendation Results

[((80937808, 386360), 0.17942816109131407),
 ((80937808, 380600), 0.05183744885757856),
 ((80937808, 315640), 0.08063279973821186),
 ((80937808, 226320), 0.11570425808599043),
 ((80937808, 339610), 0.09473156009508929)]

In [249]:
rates_and_preds = rddTesting.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions) 
#rates_and_preds.filter(lambda x : x[1][0] >= 10).take(100)

In [243]:
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) 
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 18.99216637537867


In [244]:
#Referenece: https://towardsdatascience.com/steam-recommendation-systems-4358917288eb
#https://github.com/Akxay/recommendation_engine/blob/master/Jobs_RE_spark.ipynb